In [1]:
import io
import sqlite3
import pandas as pd

In [7]:
!wget --quiet -P /tmp/ibd https://raw.githubusercontent.com/abnermneves/ibd/master/tp1/despesas.sql

In [13]:
conn = sqlite3.connect('/git/ibd/tp1/despesas.db')
cursor = conn.cursor()

In [15]:
f = io.open('/git/ibd/tp1/despesas.sql', 'r', encoding='UTF-8')
sql = f.read()
cursor.executescript(sql)

OperationalError: table [ORGAO] already exists

In [16]:
despesa = pd.read_sql_query('SELECT * FROM DESPESA_PUBLICA', conn)
orgao = pd.read_sql_query('SELECT * FROM ORGAO', conn)
programa = pd.read_sql_query('SELECT * FROM PROGRAMA_GOVERNO', conn)
funcao = pd.read_sql_query('SELECT * FROM FUNCAO', conn)
detalhamento = pd.read_sql_query('SELECT * FROM DETALHAMENTO_FUNCAO', conn)
subfuncao= pd.read_sql_query('SELECT * FROM SUBFUNCAO', conn)

<h1>Consulta 1</h1>

In [56]:
c1 = pd.read_sql_query("SELECT COD_SUBFUNCAO, NOME_SUBFUNCAO FROM DETALHAMENTO_FUNCAO NATURAL JOIN FUNCAO NATURAL JOIN SUBFUNCAO WHERE COD_FUNCAO = (SELECT CF FROM (SELECT *, MAX(QNTDSF) FROM (SELECT COD_FUNCAO AS CF, COUNT() AS QNTDSF FROM DETALHAMENTO_FUNCAO as DF GROUP BY CF)))", conn)
c1

,COD_SUBFUNCAO,NOME_SUBFUNCAO
0,125,Normatização e fiscalização
1,126,Tecnologia da informação
2,131,Comunicação social
3,244,Assistência comunitária
4,306,Alimentação e nutrição
5,365,Educação infantil
6,511,Saneamento básico rural


<h1>Consulta 6</h1>

In [55]:
c6 = pd.read_sql_query("select NOME_FUNCAO, NOME_SUBFUNCAO, SUM(VALOR_DESPESA) as VALOR_TOTAL from FUNCAO natural join DETALHAMENTO_FUNCAO natural join SUBFUNCAO natural join DESPESA_PUBLICA as DP natural join PROGRAMA_GOVERNO as PG where PG.NOME_PROGRAMA_GOVERNO = 'MAIS MEDICOS' and ANO_DESPESA = 2018 and MES_DESPESA < 10 and MES_DESPESA > 6 group by COD_SUBFUNCAO", conn)
c6

,NOME_FUNCAO,NOME_SUBFUNCAO,VALOR_TOTAL
0,Defesa nacional,Administração geral,0
1,Saúde,Atenção básica,11000
2,Educação,Assistência hospitalar e ambulatorial,0
3,Educação,Ensino superior,2400


<h1>Consulta 5</h1>

In [71]:
c5 = pd.read_sql_query("select COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO from (select *, SUM(VALOR_DESPESA) as DESPESAS_TOTAIS from FUNCAO natural join DETALHAMENTO_FUNCAO natural join DESPESA_PUBLICA natural join PROGRAMA_GOVERNO where NOME_FUNCAO = 'Educação' and ANO_DESPESA = 2018 group by COD_PROGRAMA_GOVERNO order by DESPESAS_TOTAIS desc limit 3)", conn)
c5

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,11,ENEM
1,14,PRONATEC
2,35,MAIS MEDICOS


In [39]:
programa

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,1,VIVER SEM LIMITE
1,2,SOS EMERGENCIA
2,3,MELHOR EM CASA
3,4,UBS
4,5,"UPA (INVESTIMENTO, CUSTEIO E COMPLEXO DE REGUL..."
5,6,REDE CEGONHA
6,7,CONTROLE DO CANCER
7,8,CIENCIA SEM FRONTEIRAS
8,9,CRIMINALIDADE VIOLENTA
9,11,ENEM


In [41]:
despesa

,COD_DESPESA,COD_ORGAO,COD_PROGRAMA_GOVERNO,COD_DETALHAMENTO,ANO_DESPESA,MES_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,0,20000,24,8,2018,11,Reserva de Contingência,0.0
1,1,20000,24,10,2018,11,Reserva de Contingência,0.0
2,2,20000,24,10,2018,12,Reserva de Contingência,0.0
3,3,20000,24,11,2018,3,Reserva de Contingência,0.0
4,4,20000,24,11,2018,4,Reserva de Contingência,0.0
5,5,20000,24,11,2018,5,Reserva de Contingência,0.0
6,6,20000,24,11,2018,7,Reserva de Contingência,0.0
7,7,20000,24,11,2018,11,Reserva de Contingência,0.0
8,8,20000,29,4,2018,10,Reserva de Contingência,0.0
9,9,20000,29,4,2018,12,Reserva de Contingência,0.0


In [18]:
subfuncao

,COD_SUBFUNCAO,NOME_SUBFUNCAO
0,122,Administração geral
1,125,Normatização e fiscalização
2,126,Tecnologia da informação
3,127,Ordenamento territorial
4,128,Formação de recursos humanos
5,131,Comunicação social
6,153,Defesa terrestre
7,181,Policiamento
8,182,Defesa civil
9,183,Informação e inteligência


In [19]:
funcao

,COD_FUNCAO,NOME_FUNCAO
0,3,Essencial à justiça
1,5,Defesa nacional
2,6,Segurança pública
3,8,Assistência social
4,10,Saúde
5,11,Trabalho
6,12,Educação
7,14,Direitos da cidadania
8,15,Urbanismo
9,18,Gestão ambiental


In [20]:
detalhamento

,COD_DETALHAMENTO,COD_FUNCAO,COD_SUBFUNCAO
0,0,3,131
1,1,5,122
2,2,5,153
3,3,6,122
4,4,6,181
5,5,6,182
6,6,6,183
7,7,8,125
8,8,8,126
9,9,8,131
